In [1]:
import pandas as pd
import datetime
import numpy as np
import os.path
import wget
from zipfile import ZipFile
import shutil
import re
from multiprocessing import Pool
import multiprocessing as mp

In [2]:
## Variaveis globais

#qual base de dados quero usar, "con" para consolidado e "ind" para individual
tipo = "con"

#Pega data corrente e ano corrente
date = datetime.date.today()
year = date.year

# ano inicial pra coleta dos dados
year_0 = 2010 
year_0_tri = year-2 # ano inicial pra coletoa dos dados trimestrais

In [3]:
# baixa todos os dados se não estiverem presentes, retorna uma lista de nomes de arquivos baixados
def baixa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    # URL's onde estão os arquivos dfp e itr na CVM
    url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
    url_base_fre = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    url_base_tri = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    url_base_fca = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/"

    # Cria uma lista com os nomes dos arquivos de dados anuais (dfp) da CVM
    arquivos_zip = []
    for ano in range(year_0, year + 1):
        arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

    # baixa os arquivos dfp
    arquivos_zip_dfp_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_dfp_baixados.append(arq)
        else:
            try:
                wget.download(url_base + arq, out=f"./Data/{arq}")
                arquivos_zip_dfp_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # Cria lista de arquivos ITR (trimestrais) pra baixar
    arquivos_zip = []
    for ano in range(year_0_tri, year + 1):
        arquivos_zip.append(f'itr_cia_aberta_{ano}.zip')

    # baixa os arquivos ITR (dados trimestrais) se não estiverem presentes
    arquivos_zip_itr_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_itr_baixados.append(arq)
        else:
            try:
                wget.download(url_base_tri + arq, out=f"./Data/{arq}")
                arquivos_zip_itr_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # baixa arquivo FCA, que contém os tickers

    # Cria lista de arquivos FCA  pra baixar. Este aqui só precisa baixar o do último ano disponível
    # Se não tiver do ano atual baixa o do ano passado
    arquivos_zip = []
    for ano in [year, year - 1]:
        arquivos_zip.append(f'fca_cia_aberta_{ano}.zip')

    arquivos_zip_fca_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fca_baixados.append(arq)
            break
        else:
            try:
                wget.download(url_base_fca + arq, out=f"./Data/{arq}")
                arquivos_zip_fca_baixados.append(arq)
                break # se conseguiu baixar 1 já sai do loop
            except:
                print(f"falha ao baixar {arq}")
    
    # baixa arquivo FRE, que contém o número total de ações
    
    arquivos_zip = []
    for ano in range(year-3, year+1):
        arquivos_zip.append(f'fre_cia_aberta_{ano}.zip')

    arquivos_zip_fre_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fre_baixados.append(arq)
        else:
            try:
                wget.download(url_base_fre + arq, out=f"./Data/{arq}")
                arquivos_zip_fre_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                
    return (arquivos_zip_dfp_baixados, arquivos_zip_itr_baixados, arquivos_zip_fca_baixados, arquivos_zip_fre_baixados)

In [4]:
def processa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    print("Processando dados")
    
    # DFP - Dados anuais
    # junta todos os anos de dados anuais em um arquivo só de cada tipo
    nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
             'DFC_MD_con', 'DFC_MI_con', 'DFC_MD_ind', 'DFC_MI_ind']
    for nome in nomes:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0, year):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/dfp_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em {nome}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/dfp_cia_aberta_{nome}_{year_0}-{year}.csv', index=False, encoding="latin1")
        print(f'DFP {nome} processado')
    
    # ITR - Dados trimestrais
    # junta todos os anos de dados trimestrais em um arquivo só de cada tipo
    nomes_tri = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
                 "DFC_MD_con", "DFC_MI_con", "DFC_MD_ind", "DFC_MI_ind"]
    for nome in nomes_tri:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0_tri, year + 1):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/itr_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em itr_cia_aberta_{nome}_{ano}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/itr_cia_aberta_{nome}_{year_0_tri}-{year}.csv', index=False, encoding="latin1")
        print(f'ITR {nome} processado')
    
    # FRE - Contém q qtd de ações das empresas
    
    # cria uma lista com todos os arquivos fre
    arq_fre = [item for item in os.listdir("./Data/temp") if item.startswith("fre_cia_aberta_distribuicao_capital_2")]
    fre = pd.DataFrame()

    # concatena tudo
    for arq in arq_fre[::-1]:
        fre = pd.concat([fre, pd.read_csv(f"./Data/temp/{arq}", sep=";")], axis=0)

    # joga fora as linhas com 0
    fre = fre[fre["Quantidade_Total_Acoes_Circulacao"] != 0]

    # ordena pelo mais recente, retira as duplicatas e pega só as colunas que interessam
    fre = fre.sort_values(by=["Data_Referencia", "Versao"], ascending=False).drop_duplicates(["CNPJ_Companhia"], keep="first")[["CNPJ_Companhia", "Quantidade_Total_Acoes_Circulacao"]]

    # salva em um csv
    fre.to_csv("./Data/processados/qtd_acoes.csv", encoding="latin1", index=False)
    #fre.set_index("CNPJ_Companhia", inplace=True)
    print("FRE processado")
    
    # arquivo FCA - Contém tickers das empresas
    
    # pega o nome do arquivo fca que contem os tickers
    arq_fca = [item for item in os.listdir("./Data/temp") if item.startswith("fca_cia_aberta_valor_mobiliario")]
    arq_fca = arq_fca[0]

    # copia o arquivo fca pra pasta processados
    shutil.copyfile(f"./Data/temp/{arq_fca}", f"./Data/processados/fca.csv")
    print("FCA processado")
        
    print("Arquivos processados")

In [5]:
# só extrai o arquivo dado como parâmetro para a pasta ./Data/temp
def extrai_arquivo(arq):
    try:
        ZipFile(f"./Data/{arq}", 'r').extractall('./Data/temp')
        print(f"{arq} extraido")
    except:
        print(f"erro ao extrair {arq}")

# extrai uma lista de arquivos com paralelismo. não roda no jupyter

def extrai_paralelo(arquivos_para_extrair):

    #  descobre quantas threads tem o CPU atual e cria uma pool para extrair os dados com paralelismo
    procs = mp.cpu_count()
    pool = Pool(procs)

    # cria uma lista de pools para depois dar get
    lista_pool = []
    for arq in arquivos_para_extrair:
        lista_pool.append(pool.apply_async(extrai_arquivo, args=(arq,)))
    print("Extraindo")
    output = [item.get() for item in lista_pool]
    return(output)

In [6]:
# faz o usuario escolher uma empresa
def escolhe_empresa(lista_empresas):
    while True:
        busca = input("Entre com o ticker da empresa sem o número: ")
        empresa = lista_empresas[lista_empresas['TICKER'].str.contains(busca, case=False)]
        print(empresa[["DENOM_CIA", "CD_CVM", "TICKER"]])
        ok = input("A empresa desejada está na primeira linha? (y/n) ")
        if ok == "y":
            return(empresa)

In [110]:
# retorna um df com CNPJ razao social e ticker de todas as empresas listadas em bolsa
def cria_lista_empresas(year_0=year_0, year=year, year_0_tri=year_0_tri):

    #cria lista de empresas. Concatena consolidado e individual
    lista_empresas = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_con_{year_0}-{year}.csv", header=0, sep=",", encoding="latin1")[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    lista_empresas["tipo"] = "con"
    ind = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_ind_{year_0}-{year}.csv", header=0, sep=",", encoding="latin1")[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    ind["tipo"] = "ind"
    lista_empresas = pd.concat([lista_empresas, ind], ignore_index=True).drop_duplicates(keep="first")
    lista_empresas.set_index("CNPJ_CIA", inplace=True)

    # coloca tickers do arquivo fca
    lista_empresas = lista_empresas.join(pd.read_csv(f"./Data/processados/fca.csv", header=0, sep=";", encoding="latin1").set_index("CNPJ_Companhia")["Codigo_Negociacao"].dropna())

    # pega tickers da B3 tbm pq na CVM é incompleto.
    # extraí de https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm
    # no futuro seria bom automatizar isso com um webscraper
    b3 = pd.read_csv("./Data/empresas_listadas_b3.csv", encoding="latin1", sep=";")[["Razão Social","Código"]]

    # Cria a coluna CNPJ com os dados do index
    lista_empresas = lista_empresas.rename_axis('CNPJ').reset_index()

    # Junta os dados da b3 no arquivo original
    lista_empresas = lista_empresas.merge(b3, left_on="DENOM_CIA", right_on="Razão Social", how="left").drop(columns=["Razão Social"])
    
    # Dá replace em todas as linhas de "Codigo_Negociacao" que não obedecem o padrão "ABCD1" ou "ABCD11"
    lista_empresas["Codigo_Negociacao"].replace(to_replace= r"^(?!\D{4}\d{1,2}$).*", value= np.nan, regex=True, inplace=True)

    # Pega só as 4 primeiras letras da coluna "Codigo_Negociacao"
    lista_empresas["TICKER"] = lista_empresas["Codigo_Negociacao"].str[:4]
    lista_empresas.drop(columns=["Codigo_Negociacao"], inplace=True)
    lista_empresas.drop_duplicates(inplace=True)

    # Dá replace nos NaN da CVM com o ticker do site da B3
    lista_empresas["TICKER"].fillna(lista_empresas["Código"], inplace=True)

    # Joga fora a coluna com tickers da B3
    lista_empresas.drop(columns=["Código"], inplace=True)
    
    # joga fora o que não tiver ticker
    lista_empresas.dropna(inplace=True)
    
    #deixa o index como o CNPJ
    lista_empresas.set_index("CNPJ")
    
    # tira linhas duplicadas (tem que ter CNPJ, CD_CVM e TICKER iguais)
    lista_empresas.drop_duplicates(["CNPJ", "CD_CVM", "TICKER"], inplace=True)
    
    # muda o index pro CNPJ pra poder dar join
    lista_empresas.set_index("CNPJ", inplace=True)

    # pega a quantidade de ações em circulação e tira as empresas que não tem nenhuma. isso limpa bem os dados
    fre = pd.read_csv("./Data/processados/qtd_acoes.csv").set_index("CNPJ_Companhia")
    lista_empresas = lista_empresas.join(fre).dropna()
    
    #salva em um csv
    lista_empresas.to_csv("./Data/processados/lista empresas.csv", encoding="latin1", index=False)
    
    return(lista_empresas)

In [129]:
# funcao colocar lucros ultimos 10 anos na lista_empresas e calcular indicadores
def Coloca_lucro(lista_empresas, year_0=year_0, year=year, year_0_tri=year_0_tri):

    # processa separado pra empresas consolidadas e individuais
    lista_empresas_con = lista_empresas[lista_empresas["tipo"] == "con"]
    
    # cria df dre pros dados consolidado
    tipo= "con"
    dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
                 pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
               ignore_index=True).drop_duplicates()
    dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
    dre.set_index("CNPJ_CIA", inplace=True)
    
    # coloca as colunas de lucro com o nome da coluna correspondendo ao trimestre/ano e calcula o 4T
    for ano in range(year_0, year+1):
        for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
            lista_empresas_con = lista_empresas_con.join(dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
                                                             (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
                                                             (((dre.DS_CONTA == 'Lucro/Prejuízo Consolidado do Período')) | (dre.DS_CONTA == 'Lucro ou Prejuízo Líquido Consolidado do Período'))][
                ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1))

        lista_empresas_con[f"4T{ano}"] = lista_empresas_con[f"{ano}"] - lista_empresas_con[f"3T{ano}"] - lista_empresas_con[f"2T{ano}"] - lista_empresas_con[f"1T{ano}"]
    lista_empresas_con.dropna(axis=1, inplace=True, how="all")
    
    # tira o cnpj do index pra poder rodar em todas as linhas
    lista_empresas_con = lista_empresas_con.rename_axis('CNPJ').reset_index()
    
    # cria coluna TTM (últimos 12 meses)
    #Para todas as linhas:
    for linha in lista_empresas_con.index:
        # pega os nomes das últimas 4 colunas que são trimestrais, contém dados e são do ano atual ou do último
        ultimos_4_tri = [item for item in list(lista_empresas_con.loc[linha].dropna().index) if (bool(re.search(r"^(\dT)", item)) and ((item.endswith(str(year))) or (item.endswith(str(year-1)))))][-4:]
        # se tiver 4 trimestres, soma e põe na caoluna TTM
        if len(ultimos_4_tri) == 4:
            lista_empresas_con.loc[linha, "TTM"] = lista_empresas_con.loc[linha, ultimos_4_tri].sum()
    
    # dropa todas as colunas trimestrais
    colunas_tri = [item for item in list(lista_empresas_con.columns) if (bool(re.search(r"^(\dT)", item)))]
    lista_empresas_con.drop(columns=colunas_tri, inplace=True)

    # ############################################################################################## 
    # agora faz pro individual
    
    # processa separado pra empresas consolidadas e individuais
    lista_empresas_ind = lista_empresas[lista_empresas["tipo"] == "ind"]
    
    # cria df dre pros dados consolidado
    tipo= "ind"
    dre = pd.concat([pd.read_csv(f'./Data/processados/dfp_cia_aberta_DRE_{tipo}_{year_0}-{year}.csv', encoding="latin1"),
                 pd.read_csv(f'./Data/processados/itr_cia_aberta_DRE_{tipo}_{year_0_tri}-{year}.csv', encoding="latin1")],
               ignore_index=True).drop_duplicates()
    dre = dre[dre["ORDEM_EXERC"] == "ÚLTIMO"]
    dre.set_index("CNPJ_CIA", inplace=True)
    
    # coloca as colunas de lucro com o nome da coluna correspondendo ao trimestre/ano e calcula o 4T
    for ano in range(year_0, year+1):
        for tri_ini, tri_fim, nome_tri in zip(["01", "04", "07", "10", "01"], ["03", "06", "09", "12", "12"], ["1T", "2T", "3T", "4T", ""]):
            lista_empresas_ind = lista_empresas_ind.join(dre[(dre.DT_INI_EXERC == (f"{ano}-{tri_ini}-01")) & 
                                                             (dre.DT_FIM_EXERC.str.contains(f"{ano}-{tri_fim}-3")) & 
                                                             ((dre.DS_CONTA == 'Lucro/Prejuízo do Período') | ((dre.DS_CONTA == 'Lucro ou Prejuízo Líquido do Período')))][
                ["VL_CONTA", "DS_CONTA"]].rename(columns={"VL_CONTA": f"{nome_tri}{ano}"}).drop("DS_CONTA", axis=1))

        lista_empresas_ind[f"4T{ano}"] = lista_empresas_ind[f"{ano}"] - lista_empresas_ind[f"3T{ano}"] - lista_empresas_ind[f"2T{ano}"] - lista_empresas_ind[f"1T{ano}"]
    lista_empresas_ind.dropna(axis=1, inplace=True, how="all")
    
    # tira o cnpj do index pra poder rodar em todas as linhas
    lista_empresas_ind = lista_empresas_ind.rename_axis('CNPJ').reset_index()
    
    # cria coluna TTM (últimos 12 meses)
    #Para todas as linhas:
    for linha in lista_empresas_ind.index:
        # pega os nomes das últimas 4 colunas que são trimestrais, contém dados e são do ano atual ou do último
        ultimos_4_tri = [item for item in list(lista_empresas_ind.loc[linha].dropna().index) if (bool(re.search(r"^(\dT)", item)) and ((item.endswith(str(year))) or (item.endswith(str(year-1)))))][-4:]
        # se tiver 4 trimestres, soma e põe na coluna TTM
        if len(ultimos_4_tri) == 4:
            lista_empresas_ind.loc[linha, "TTM"] = lista_empresas_ind.loc[linha, ultimos_4_tri].sum()
    
    # dropa todas as colunas trimestrais
    colunas_tri = [item for item in list(lista_empresas_ind.columns) if (bool(re.search(r"^(\dT)", item)))]
    lista_empresas_ind.drop(columns=colunas_tri, inplace=True)
    
    # junta con e ind
    return(pd.concat([lista_empresas_con, lista_empresas_ind], axis=0, ignore_index=True).set_index("CNPJ").drop_duplicates())

In [130]:
# ve se o lucro dos últimos 10 anos foi maior que 0 e se o crescimento foi maior que 30%. Coloca em 2 novas colunas
# dá NaN quando não tem dados suficientes pra calcular, 1 pra quando passa no filtro e 0 pra quando não passa
def Indicadores_lucro(lista_empresas):
    # pra todas as linhas
    for linha in lista_empresas.index:
        # Faz uma lista com os últimos 10 anos que contém dados (contando com TTM)
        ultimos_10_anos = [item for item in list(lista_empresas.loc[linha].dropna().index) if (bool(re.search(r"^(\d{4})", item)) or (item == "TTM"))][-10:]
        # se tiver 10 anos de dados
        if (len(ultimos_10_anos) == 10):
            # e o lucro for positivo pra todos os elementos da linha
            if ((lista_empresas.loc[linha, ultimos_10_anos] > 0).all()):
                # a coluna "Sem prejuízo nos últimos 10 anos" recebe 1,
                lista_empresas.loc[linha, "Sem prejuízo nos últimos 10 anos"] = 1
            else:
                #  se não, 0
                lista_empresas.loc[linha, "Sem prejuízo nos últimos 10 anos"] = 0

            # Filtro de crescimento de 30% do lucro nos últimos 10 anos
            # se o lucro do ultimo ano/ 10 anos atras for maior ou igual a 30%
            if (lista_empresas.loc[linha, ultimos_10_anos[-1]]/lista_empresas.loc[linha, ultimos_10_anos[0]]) >= 1.3:
                lista_empresas.loc[linha, "Crescimento de 30% dos lucros nos últimos 10 anos"] = 1
            else:
                lista_empresas.loc[linha, "Crescimento de 30% dos lucros nos últimos 10 anos"] = 0
    return(lista_empresas)

## Main

In [9]:
# Limpeza

# Deleta o conteúdo da pasta processados se existir
lista_arquivos = os.listdir("./Data/processados/")
lista_arquivos.remove(".gitignore")

for item in lista_arquivos:
    try:
        os.remove(f"./Data/processados/{item}")
    except:
        pass

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

In [10]:
# baixa os dados e coloca os nomes dos arquivos baixados em cada variavel
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre = baixa_dados(year_0, year, year_0_tri)
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre

100% [..........................................................................] 9519100 / 9519100

(['dfp_cia_aberta_2010.zip',
  'dfp_cia_aberta_2011.zip',
  'dfp_cia_aberta_2012.zip',
  'dfp_cia_aberta_2013.zip',
  'dfp_cia_aberta_2014.zip',
  'dfp_cia_aberta_2015.zip',
  'dfp_cia_aberta_2016.zip',
  'dfp_cia_aberta_2017.zip',
  'dfp_cia_aberta_2018.zip',
  'dfp_cia_aberta_2019.zip',
  'dfp_cia_aberta_2020.zip',
  'dfp_cia_aberta_2021.zip',
  'dfp_cia_aberta_2022.zip'],
 ['itr_cia_aberta_2020.zip',
  'itr_cia_aberta_2021.zip',
  'itr_cia_aberta_2022.zip'],
 ['fca_cia_aberta_2022.zip'],
 ['fre_cia_aberta_2019.zip',
  'fre_cia_aberta_2020.zip',
  'fre_cia_aberta_2021.zip',
  'fre_cia_aberta_2022.zip'])

In [11]:
# extrai arquivos
arquivos_para_extrair = arquivos_dfp + arquivos_itr + arquivos_fca + arquivos_fre

# extrai sequencial
for arq in arquivos_para_extrair:
    extrai_arquivo(arq)
print("Pronto")
# extrai paralelo, não funciona no jupyter
#extrai_paralelo(arquivos_para_extrair)

dfp_cia_aberta_2010.zip extraido
dfp_cia_aberta_2011.zip extraido
dfp_cia_aberta_2012.zip extraido
dfp_cia_aberta_2013.zip extraido
dfp_cia_aberta_2014.zip extraido
dfp_cia_aberta_2015.zip extraido
dfp_cia_aberta_2016.zip extraido
dfp_cia_aberta_2017.zip extraido
dfp_cia_aberta_2018.zip extraido
dfp_cia_aberta_2019.zip extraido
dfp_cia_aberta_2020.zip extraido
dfp_cia_aberta_2021.zip extraido
dfp_cia_aberta_2022.zip extraido
itr_cia_aberta_2020.zip extraido
itr_cia_aberta_2021.zip extraido
itr_cia_aberta_2022.zip extraido
fca_cia_aberta_2022.zip extraido
fre_cia_aberta_2019.zip extraido
fre_cia_aberta_2020.zip extraido
fre_cia_aberta_2021.zip extraido
fre_cia_aberta_2022.zip extraido


In [14]:
# concatena os arquivos baixados do ano inicial até o final
processa_dados(year_0, year, year_0_tri)

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

Processando dados
DFP BPA_con processado
DFP BPA_ind processado
DFP BPP_con processado
DFP BPP_ind processado
DFP DRE_con processado
DFP DRE_ind processado
DFP DFC_MD_con processado
DFP DFC_MI_con processado
DFP DFC_MD_ind processado
DFP DFC_MI_ind processado
ITR BPA_con processado
ITR BPA_ind processado
ITR BPP_con processado
ITR BPP_ind processado
ITR DRE_con processado
ITR DRE_ind processado
ITR DFC_MD_con processado
ITR DFC_MI_con processado
ITR DFC_MD_ind processado
ITR DFC_MI_ind processado
FRE processado
FCA processado
Arquivos processados


## Fim do processamento dos dados

In [124]:
# cria um dataframe com todas as empresas listadas, tipo de balanço, ticker e ações em circulação Já vem bastante limpo.
lista_empresas = cria_lista_empresas(year_0, year, year_0_tri)
#lista_empresas.to_excel("teste2.xlsx")
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08
...,...,...,...,...,...
94.813.102/0001-70,TRÊS TENTOS AGROINDUSTRIAL S/A,25950,ind,TTEN,1.123436e+08
95.426.862/0001-97,EXCELSIOR ALIMENTOS S.A.,1570,ind,BAUH,1.508636e+06
96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,con,LJQQ,1.789674e+08
97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,con,ODER,1.979520e+05


In [ ]:
# calcular indicadores e colocar cotação (adiar ao maximo pq demora mto tempo pra pegar)

In [127]:
lista_empresas = Coloca_lucro(lista_empresas)
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,TTM
CNPJ,,,,,,,,,,,,,,,,,,
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09,11330345.0,12736912.0,11438200.0,11288834.0,13343496.0,15798039.0,8659577.0,12275303.0,15086101.0,18888318.0,13292883.0,19722871.0,NaN
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07,262184.0,128936.0,195644.0,211225.0,226876.0,151027.0,170443.0,219070.0,324211.0,417146.0,NaN,NaN,NaN
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08,2552985.0,3762019.0,-6925652.0,-6291375.0,-2962502.0,-14953658.0,3513276.0,-1725691.0,13347577.0,10743843.0,6387313.0,5713633.0,6820869.0
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07,19617.0,46132.0,74679.0,-75163.0,-96319.0,88519.0,133693.0,177233.0,115710.0,156629.0,465378.0,1294407.0,196629.0
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58064.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91.669.747/0001-92,FINANSINOS S.A.- CREDITO FINANC E INVEST,6076,ind,FNCN,3.001010e+05,453.0,631.0,1394.0,462.0,681.0,-4539.0,-128.0,375.0,-2323.0,-627.0,-2088.0,-1143.0,-859.0
92.749.225/0001-63,HERCULES S.A. FABRICA DE TALHERES,6629,ind,HETA,2.437500e+05,-16586.0,-36357.0,-31366.0,-33866.0,80117.0,-6738.0,3045.0,-52508.0,1344.0,3436.0,-172049.0,443.0,-56.0
93.828.986/0001-73,CEMEPE INVESTIMENTOS S.A.,13854,ind,MAPT,3.569300e+04,-111.0,-378.0,-143.0,-133.0,-138.0,-209.0,-200.0,-141.0,-176.0,-163.0,-170.0,-178.0,-172.0


In [128]:
# coloca os indicadores  na lista
lista_empresas = Indicadores_lucro(lista_empresas)
lista_empresas

,DENOM_CIA,CD_CVM,tipo,TICKER,Quantidade_Total_Acoes_Circulacao,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,TTM,Sem prejuízo nos últimos 10 anos,Crescimento de 30% dos lucros nos últimos 10 anos
CNPJ,,,,,,,,,,,,,,,,,,,,
00.000.000/0001-91,BCO BRASIL S.A.,1023,con,BBAS,1.420731e+09,11330345.0,12736912.0,11438200.0,11288834.0,13343496.0,15798039.0,8659577.0,12275303.0,15086101.0,18888318.0,13292883.0,19722871.0,NaN,1.0,1.0
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,con,BSLI,4.195216e+07,262184.0,128936.0,195644.0,211225.0,226876.0,151027.0,170443.0,219070.0,324211.0,417146.0,NaN,NaN,NaN,1.0,1.0
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,con,ELET,5.997421e+08,2552985.0,3762019.0,-6925652.0,-6291375.0,-2962502.0,-14953658.0,3513276.0,-1725691.0,13347577.0,10743843.0,6387313.0,5713633.0,6820869.0,0.0,0.0
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,con,CEBR,1.430870e+07,19617.0,46132.0,74679.0,-75163.0,-96319.0,88519.0,133693.0,177233.0,115710.0,156629.0,465378.0,1294407.0,196629.0,0.0,0.0
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,con,ARML,1.718094e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58064.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91.669.747/0001-92,FINANSINOS S.A.- CREDITO FINANC E INVEST,6076,ind,FNCN,3.001010e+05,453.0,631.0,1394.0,462.0,681.0,-4539.0,-128.0,375.0,-2323.0,-627.0,-2088.0,-1143.0,-859.0,0.0,0.0
92.749.225/0001-63,HERCULES S.A. FABRICA DE TALHERES,6629,ind,HETA,2.437500e+05,-16586.0,-36357.0,-31366.0,-33866.0,80117.0,-6738.0,3045.0,-52508.0,1344.0,3436.0,-172049.0,443.0,-56.0,0.0,0.0
93.828.986/0001-73,CEMEPE INVESTIMENTOS S.A.,13854,ind,MAPT,3.569300e+04,-111.0,-378.0,-143.0,-133.0,-138.0,-209.0,-200.0,-141.0,-176.0,-163.0,-170.0,-178.0,-172.0,0.0,0.0


In [ ]:
# Futuro modulo de gerar relatório
empresa = escolhe_empresa(lista_empresas)
cnpj_selecionado = empresa.index[0]
tipo_empresa_selecionada = empresa["tipo"].iloc[0]
cnpj_selecionado, tipo_empresa_selecionada